# Dataset Analysis and Curation

This is from from This guy's notebook [here](https://www.kaggle.com/code/maxbr0wn/fine-tuning-xtts-v2-english?scriptVersionId=173570803) and Coqui's TTS notebook [here](https://github.com/coqui-ai/TTS/blob/dev/notebooks/dataset_analysis/AnalyzeDataset.ipynb)

# The only input you'll be required to give is the path to the dataset folder. Everything else should be automated. Including the curation/removal of poor quality samples.

# Use this notebook with the provided known good dataset first. Then you can compare your own dataset.

You will need to combine your train_list.txt and val_list.txt into a single "metadata.csv" file. The csv needs to be formatted without a header and in this order --> 
audio_file_name|transcription|transcription
# Dont use the IPA translation.

**Example:**
audio_000001|It contained a bookcase: I soon possessed myself of a volume, taking care that it should be one stored with pictures.|It contained a bookcase: I soon possessed myself of a volume, taking care that it should be one stored with pictures.

You can continue through each cell with the example dataset until it tells you to stop. Theres no need to do the cells beyond that point.

When you are ready to use your own dataset youll have the option to remove the poor quality samples and regenerate the csv. At that point you can recreate the train_list and val_list.txt and phonemize.

# The cell below is to grab the reference good quality dataset. It can be skipped without any other modifications.

In [36]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'janeeyre:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4372173%2F7507317%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240608%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240608T194313Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D21443d92e1d8791f65ec5a33925c559685283f40417a61348248dc6fd8f18fd65f08c2ec964023417fa73c7ab98a3ed5a8f82384cef647c396805c1a8244545c879a5f38fe2724f09fe8a2a0046be8cdce398fca60e3aa2698fee1df45a19ee7834fd4beb4aa6d5db4a3bdc5637910b3deefcff20f682a57f4eaedc7d827b000fa15431ca593453f1da86dfe99b53d7ac8eeb8e4ac3b856b033147cabc3a09bccb994f8e02bbb15ef21970333a4e904b1b4db235fb5dbe8ecc09a5126e402b196ad6e8a06067950f34f40f6a804dfbd45434b01fed82ac79d8cb06ec8455e5d0946b00fcc396d7d4a19cb63c59ee9d079e46782628548cdc2aa69e10df1c9df1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 1615805879 bytes downloaded
Downloaded and uncompressed: janeeyre
Dataset saved to: /home/eleven/dset/input/janeeyre
Data source import complete.


In [9]:
%pip install git+https://github.com/coqui-ai/TTS

# or !pip install git+https://github.com/coqui-ai/TTS

  Cloning https://github.com/coqui-ai/TTS to /tmp/pip-req-build-u_zzpk_7
  Running command git clone --filter=blob:none --quiet https://github.com/coqui-ai/TTS /tmp/pip-req-build-u_zzpk_7
  Resolved https://github.com/coqui-ai/TTS to commit dbf1a08a0d4e47fdad6172e433eeb34bc6b13b4e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for cython>=0.29.30 from https://files.pythonhosted.org/packages/45/82/077c13035d6f45d8b8b74d67e9f73f2bfc54ef8d1f79572790f6f7d2b4f5/Cython-3.0.10-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for scipy>=1.11.2 from https://files.pythonhosted.org/packages/36/07/035d22ff9795129c5a847c64cb43c1fa9188826b59344fee28a3ab02e283/scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 892.3 kB/s eta 0:00:00 0:00:01

In [10]:
import os
import sys
import csv
import shutil
import librosa
import numpy as np
import pandas as pd
from scipy.stats import norm
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool
from matplotlib import pylab as plt
from collections import Counter
from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.datasets.formatters import *
%matplotlib inline

### Load Dataset

Input path to the folder that contains the metadata.csv the wavs folder.
/path/to/your/dataset/
├── metadata.csv
└── wavs/

In [31]:
training_dir = '/kaggle/input/trmp'
NUM_PROC = 4
DATASET_CONFIG = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path=training_dir
)

In [32]:
def formatter(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    txt_file = os.path.join(root_path, meta_file)
    items = []
    with open(meta_file, "r", encoding="utf-8") as f:
        for line in f:
            print(f"Processing line: {line.strip()}")  # Debugging statement
            cols = line.split("|")
            print(f"Columns: {cols}")  # Debugging statement
            if len(cols) < 2:
                print(f"Skipping line due to insufficient columns: {line.strip()}")
                continue
            wav_file = os.path.join(root_path, "wavs", cols[0] + ".wav")
            text = cols[1]
            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items



In [33]:
train_samples, eval_samples = load_tts_samples(DATASET_CONFIG, eval_split=True, formatter=formatter)
if eval_samples is not None:
    items = train_samples + eval_samples
else:
    items = train_samples

FileNotFoundError: [Errno 2] No such file or directory: 'metadata.csv'

In [ ]:
# How many audio files are there in the dataset?
print(" > Number of audio files: {}".format(len(items)))
print(items[0])

In [ ]:
# print out names of missing wavs
wav_files = []
for item in items:
    wav_file = item["audio_file"].strip()
    wav_files.append(wav_file)
    if not os.path.exists(wav_file):
        print(wav_file)

In [ ]:
# print out names of duplicate wavs
c = Counter(wav_files)
print([item for item, count in c.items() if count > 1])

### Compute Audio/Text Lengths

In [ ]:

def load_item(item):
    text = item["text"].strip()
    file_name = item["audio_file"].strip()
    audio, sr = librosa.load(file_name, sr=None)
    audio_len = len(audio) / sr
    text_len = len(text)
    return file_name, text, text_len, audio, audio_len

# This will take a while depending on size of dataset
if NUM_PROC == 1:
    data = []
    for m in tqdm(items):
        data += [load_item(m)]
else:
    with Pool(NUM_PROC) as p:
        data = list(tqdm(p.imap(load_item, items), total=len(items)))

In [ ]:
# look at breadth of vocabulary
w_count = Counter()
for item in tqdm(data):
    text = item[1].lower().strip()
    for word in text.split():
        w_count[word] += 1
print(" > Number of words: {}".format(len(w_count)))

In [ ]:
#length of text, duration of audio in seconds and compare
text_vs_durs = {}  # text length vs audio duration
text_len_counter = Counter()  # number of sentences with the keyed length
lengths = []
for item in tqdm(data):
    text = item[1].lower().strip()
    text_len = len(text)
    text_len_counter[text_len] += 1
    lengths.append(text_len)
    audio_len = item[-1]
    try:
        text_vs_durs[text_len] += [audio_len]
    except:
        text_vs_durs[text_len] = [audio_len]


In [ ]:
#for each bucket of char lengths, find averages for that length vs duration of audio
text_vs_avg = {}
text_vs_median = {}
text_vs_std = {}
for key, durs in text_vs_durs.items():
    text_vs_avg[key] = np.mean(durs)
    text_vs_median[key] = np.median(durs)
    text_vs_std[key] = np.std(durs)



In [ ]:
#per char duration
sec_per_chars = []
for item in data:
    text = item[1]
    dur = item[-1]
    sec_per_char = dur / len(text)
    sec_per_chars.append(sec_per_char)

mean = np.mean(sec_per_chars)
std = np.std(sec_per_chars)
print(mean)
print(std)

### Visualise Dataset

In [ ]:
plt.title("text length vs mean audio duration")
plt.scatter(list(text_vs_avg.keys()), list(text_vs_avg.values()))

Note that "text length" is amount of chars. Audio duration should basically be a linear function of text length.

In [ ]:
plt.title("text length vs STD")
plt.scatter(list(text_vs_std.keys()), list(text_vs_std.values()))

This is showing us, for each set of data items with a particular char length, what is one standard deviation in audio duration for that set.

In [ ]:
plt.hist(lengths, bins=20, edgecolor='black')  # Adjust the number of bins as needed
plt.xlabel('Text Length')
plt.ylabel('Frequency')
plt.title('Distribution of Text Lengths in TTS Dataset')
plt.show()

The TTS docs recommends a 'Gaussian-like' distribution of clip lengths. A fairly uniform distribution should be fine too. I think the most important thing is to have examples across a range of different lengths. From my experience, the model does not generalise well across different sequence lengths (i.e., if you fine-tune with no short 1.5s clips and then try to create such a short clip at inference time, then it will probably struggle).

### Clean the Dataset

Now we will apply some filters to remove items from our dataset that are likely to be problematic

In [ ]:
cleaned_data=[]
len(data), len(cleaned_data)

In [ ]:
#first find our per char duration mean/sd
durs_per_char = []
for each in data:
    durs_per_char.append(each[-1]/each[2])
durs_mean = np.mean(durs_per_char)
durs_sd = np.std(durs_per_char)

durs_mean, durs_sd

In [ ]:
points = durs_per_char
mean = durs_mean
std_dev = durs_sd

plt.figure(figsize=(10, 8))
plt.hist(points, bins=30, edgecolor='black', linewidth=1.2)

# Add vertical lines for mean and standard deviation
plt.axvline(mean, color='white', linestyle='dashed', linewidth=1.5, label=f'Mean: {mean:.5f}')
plt.axvline(mean + std_dev, color='orange', linestyle='dashed', linewidth=1.5, label=f'Standard Deviation: {std_dev:.5f}')
plt.axvline(mean - std_dev, color='orange', linestyle='dashed', linewidth=1.5)
plt.axvline(mean + 2*std_dev, color='red', linestyle='dashed', linewidth=1.5, label=f'Standard Deviation: {std_dev:.5f}')
plt.axvline(mean - 2*std_dev, color='red', linestyle='dashed', linewidth=1.5)

# Add legend
plt.legend()

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Speech Duration Per Character for All Items')

# Show plot
plt.grid(True)
plt.show()

We have found the mean duration per character of text and the above histogram shows how our dataset is distributed around that mean.

Next we will filter out items with too short duration, items with too long duration, and items with too much variance

In [ ]:
minimum_duration = 0.7
maximum_duration = 13.0
maximum_sds = 2.5

In [ ]:
#TODO: add separate filter for removing items whose char length is > 250

cleaned_data = []

shorties = []
longies = []
misfits = []
for item in data:
    item_perchar_dur = item[-1]/item[2]
    difference = abs(item_perchar_dur-durs_mean)
    item_zscore = difference / durs_sd
    item = item + (item_zscore,) #add the zscore to the data item so we can sort by it later
    if item[-2] < minimum_duration:
        shorties.append(item)
    elif item[-2] > maximum_duration:
        longies.append(item)
    elif item_zscore > maximum_sds:
        misfits.append(item)
    else:
        cleaned_data.append(item)

excluded = shorties + longies + misfits


print(f"found {len(shorties)} short items and {len(longies)} long items and {len(misfits)} items whose length conformed but whose per-char duration exceeded {maximum_sds} standard deviations from the mean. Excluding {len(shorties)+len(longies)+len(misfits)} items")

Look at the items we are deeming too short to see if it looks sensible to exclude them

In [ ]:
ranked_shorts = sorted(shorties, key=lambda x: x[-2])
if len(ranked_shorts) > 0:
    print(f"Duration of shortest item excluded for being too short: {ranked_shorts[0][-2]} Text from shortest item excluded for being too short: {ranked_shorts[0][1]}")
    print(ranked_shorts[0])
    print(f"Duration of longest item excluded for being too short: {ranked_shorts[-1][-2]} Text from shortest item excluded for being too short: {ranked_shorts[-1][1]}")
    print(ranked_shorts[-1])


Likewise for the items with high variance (you might want to play around with the maximum_sds value above to find the right level of exclusion)

(If you are following along in the janeeyre dataset, this actually picks up some items from the dataset that were mislabelled)

In [ ]:
ranked_misfits = sorted(misfits, key=lambda x: x[-1])[::-1]
if len(ranked_misfits) > 0:
    print(f"Duration of worst item excluded for having too much variance: {ranked_misfits[0][-2]} and its text: {ranked_misfits[0][1]}")
    print(ranked_misfits[0])
    print(f"Duration of best item excluded for having too much variance: {ranked_misfits[-1][-2]} and its text: {ranked_misfits[-1][1]}")
    print(ranked_misfits[-1])


The following code is copy pasted from above--we are recomputing on our cleaned dataset to inspect it and compare

In [ ]:

text_vs_durs = {}  # text length vs audio duration
text_len_counter = Counter()  # number of sentences with the keyed length
lengths = []
for item in tqdm(cleaned_data):
    text = item[1].lower().strip()
    text_len = len(text)
    text_len_counter[text_len] += 1
    lengths.append(text_len)
    audio_len = item[-2]
    try:
        text_vs_durs[text_len] += [audio_len]
    except:
        text_vs_durs[text_len] = [audio_len]

In [ ]:
# text_len vs avg_audio_len, median_audio_len, std_audio_len
text_vs_avg = {}
text_vs_median = {}
text_vs_std = {}
for key, durs in text_vs_durs.items():
    text_vs_avg[key] = np.mean(durs)
    text_vs_median[key] = np.median(durs)
    text_vs_std[key] = np.std(durs)

In [ ]:
sec_per_chars = []
for item in cleaned_data:
    text = item[1]
    dur = item[-2]
    sec_per_char = dur / len(text)
    sec_per_chars.append(sec_per_char)
# sec_per_char /= len(cleaned_data)
# print(sec_per_char)

In [ ]:
mean = np.mean(sec_per_chars)
std = np.std(sec_per_chars)
print(mean)
print(std)

We should see less outliers in the visualisations below and a tighter distribution of audio duration per char

In [ ]:
plt.title("text length vs mean audio duration")
plt.scatter(list(text_vs_avg.keys()), list(text_vs_avg.values()))

In [ ]:
plt.title("text length vs median audio duration")
plt.scatter(list(text_vs_median.keys()), list(text_vs_median.values()))

In [ ]:
plt.title("text length vs STD")
plt.scatter(list(text_vs_std.keys()), list(text_vs_std.values()))

In [ ]:
plt.hist(lengths, bins=20, edgecolor='black')  # Adjust the number of bins as needed
plt.xlabel('Text Length')
plt.ylabel('Frequency')
plt.title('Distribution of Text Lengths in TTS Dataset')
plt.show()

In [ ]:
#after cleaning distribution
#first find our per char duration mean/sd
durs_per_char = []
for each in cleaned_data:
    durs_per_char.append(each[-2]/each[2])
durs_mean = np.mean(durs_per_char)
durs_sd = np.std(durs_per_char)

durs_mean, durs_sd

In [ ]:
points = durs_per_char
mean = durs_mean
std_dev = durs_sd

plt.figure(figsize=(10, 8))
plt.hist(points, bins=30, edgecolor='black', linewidth=1.2)

# Add vertical lines for mean and standard deviation
plt.axvline(mean, color='white', linestyle='dashed', linewidth=1.5, label=f'Mean: {mean:.5f}')
plt.axvline(mean + std_dev, color='orange', linestyle='dashed', linewidth=1.5, label=f'Standard Deviation: {std_dev:.5f}')
plt.axvline(mean - std_dev, color='orange', linestyle='dashed', linewidth=1.5)
plt.axvline(mean + 2*std_dev, color='red', linestyle='dashed', linewidth=1.5, label=f'Standard Deviation: {std_dev:.5f}')
plt.axvline(mean - 2*std_dev, color='red', linestyle='dashed', linewidth=1.5)

# Add legend
plt.legend()

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Speech Duration Per Character for All Items')

# Show plot
plt.grid(True)
plt.show()

### Visualise Cull

Check how much content we are proposing to remove from our dataset. I don't know how much data is 'enough' to train on. In my experience, 2 hours is plenty of audio duration (I trained a model on 10 hours of audio and a 2 hour subset of that data and I couldn't notice much difference between the two). The more high quality content, the better.

If you are having to cut a large amount of data at this stage, then your dataset probably isn't being created correctly to begin with. [here](https://github.com/zuverschenken/XTTSv2Scripts) is my repo showing how to create a dataset.

In [ ]:
len(excluded)

In [ ]:
len(data), len(cleaned_data), len(cleaned_data)/len(data)

In [ ]:
remaining_length = sum([each[-2] for each in cleaned_data])
excluded_length = sum([each[-2] for each in excluded])
remaining_length, excluded_length

In [ ]:

# Data for the pie chart (proportions)
sizes = [remaining_length, excluded_length]  # Example proportions, summing up to 100%

# Labels for each portion
labels = ['Remaining Data', 'Excluded Data']

# Colors for each portion
colors = ['lightblue', 'lightcoral']  # Adjusted color for 'lightred'

# Create pie chart
plt.figure(figsize=(6, 6))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
plt.title('Proportion of seconds of audio kept after cull')
plt.show()

In [ ]:
def seconds_to_hms(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    remaining_seconds = seconds % 60
    return hours, minutes, remaining_seconds

hours, minutes, remaining_seconds = seconds_to_hms(remaining_length)
print(f"Remaining length: {hours} hours, {minutes} minutes, and {remaining_seconds} seconds.")

In [ ]:
hours, minutes, remaining_seconds = seconds_to_hms(excluded_length)
print(f"Excluded length: {hours} hours, {minutes} minutes, and {remaining_seconds} seconds.")

### Check Parameter for Fine-tune

If you are using this data with XTTS-v2, you should use the maximum audio length from your dataset as max_wav_length and ensure the length of the reference audio you want to use is within the min-max range.

In [ ]:
speaker_reference = '/input/trmp/trumpbagdaddy.wav'
audio, sr = librosa.load(os.path.join(training_dir, 'wavs', speaker_reference), sr=None)


In [ ]:
rawLength = [x[-2] for x in cleaned_data]

print(f"seconds: {max(rawLength)}")
print(f"maximum audio file length: {max(rawLength)*sr}")

In [ ]:
print(f"speaker reference file length: {len(audio)}")


## No need to continue past this point with the example dataset.

### Delete Unwanted Files.

Finally we copy across the files we want to keep to /kaggle/working

In [ ]:
out_dir = '/workspace/output/' # choose an output directory

In [ ]:
excluded_files = set([each[0].split('/')[-1].split('.')[0] for each in excluded])

In [ ]:
#grab the rows we will keep
dropped = 0
sanitised_rows = []
with open(os.path.join(training_dir, 'metadata.csv'), 'r') as file:
    csv_reader = csv.reader(file, delimiter='|')
    for row in csv_reader:
        if row[0] not in excluded_files:
            sanitised_rows.append(row)
        else:
            dropped += 1

#sanity check
assert dropped == len(excluded_files)

In [ ]:
#create a new csv
with open(os.path.join(out_dir, 'metadata.csv'), 'w', encoding='utf-8') as file:
    csv_writer = csv.writer(file, delimiter = '|')
    for row in sanitised_rows:
        csv_writer.writerow(row)

**Note: copying all wavs to working dir. This might take some time**

In [ ]:
wavs_dir = os.path.join(out_dir, 'wavs')
os.makedirs(wavs_dir, exist_ok=True)

files = os.listdir(os.path.join(training_dir, 'wavs'))

for file in files:
    source_file = os.path.join(training_dir,'wavs', file)
    destination_file = os.path.join(wavs_dir, file)
    shutil.copy2(source_file, destination_file)


Delete the unwanted wavs

In [ ]:
#delete wavs
deleted_files = 0
for file_name in excluded_files:
    file_path = os.path.join(wavs_dir, file_name + '.wav')
    try:
        os.remove(file_path)
        deleted_files += 1
    except OSError as e:
        print(f'Error deleting file {file_path}: {e}')

#sanity check
assert deleted_files == dropped
print(f'deleted {deleted_files} wavs')

Create a tarball that you can download. Again, this might take a little while.

In [ ]:
!tar -czvf culled_dataset.tar.gz /workspace/output/*